In [1]:
import pymysql 
import traceback
import requests
import time
from datetime import datetime
import json

In [2]:
api_key = "e857655954f34ae188982244bbb23b21"
city_name = 'Dublin,ie'
parameters = {"q" : city_name, "appid" : api_key} 
WEATHER_URL = "http://api.openweathermap.org/data/2.5/weather" 

def create_table():
    sql = """
    CREATE TABLE IF NOT EXISTS weather(
        Clouds INTEGER,
        feels_like DOUBLE,
        humidity INTEGER,
        pressure INTEGER,
        temp DOUBLE,
        temp_max DOUBLE,
        temp_min DOUBLE,
        sunrise VARCHAR(255),
        sunset VARCHAR(255),
        visibility INTEGER,
        weather_description VARCHAR(255),
        weather_main VARCHAR(255),
        wind_deg INTEGER,
        wind_speed DOUBLE,
        dt VARCHAR(255)
    );
    """
        
    try:
        res = cursor.execute(sql)
        print('Created successfully!')
    except Exception as e:
        print(e)
        

In [3]:
def write_to_db(text):
    weather_data = json.loads(text)
    
    weather_vals = (
        str(weather_data['clouds']['all']),
        str(weather_data['main']['feels_like']),
        str(weather_data['main']['humidity']),
        str(weather_data['main']['pressure']),
        str(weather_data['main']['temp']),
        str(weather_data['main']['temp_max']),
        str(weather_data['main']['temp_min']),
        str(datetime.fromtimestamp(weather_data['sys']['sunrise'])),
        str(datetime.fromtimestamp(weather_data['sys']['sunset'])),
        str(weather_data['visibility']),
        str(weather_data['weather'][0]['description']),
        str(weather_data['weather'][0]['main']),
        str(weather_data['wind']['deg']),
        str(weather_data['wind']['speed']),
        str(datetime.fromtimestamp(weather_data['dt']))
    )
    
    sql = """INSERT INTO dbike.weather (Clouds,feels_like,humidity,pressure,
    temp,temp_max,temp_min,sunrise,sunset,visibility,weather_description,
    weather_main,wind_deg,wind_speed,dt) 
    VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s',
    '%s','%s','%s','%s','%s')""" % weather_vals
    
    try:
        cursor.execute(sql)
        db.commit()
        print("Insert successfully")
    except:
        db.rollback()
        print("Insert wrong")
    
    db.close()

In [ ]:
DATABASE_URL="dbike.cvo8g1gt1fco.eu-west-1.rds.amazonaws.com"
PORT= 3306
DB = "dbike"
USER = "group15"
PASSWORD = "declanmingbo"
while True:
    try:
        db = pymysql.connect(
            host=DATABASE_URL,
            user=USER,
            password=PASSWORD,
            port=PORT,
            database=DB)
        cursor = db.cursor()  
            
        create_table()

        now = datetime.now()
        r = requests.get(WEATHER_URL, params = parameters)
        print(r,now)
            
        write_to_db(r.text)
            
        time.sleep(60*60)
        
    except:
        print(traceback.format_exc())    

Created successfully!
<Response [200]> 2022-04-15 12:59:24.882131
Insert successfully
Traceback (most recent call last):
  File "/var/folders/9r/pt_53l_55hnfbqs4389kw3h40000gn/T/ipykernel_2708/2708157239.py", line 24, in <module>
    time.sleep(5*60)
KeyboardInterrupt

Created successfully!
Traceback (most recent call last):
  File "/var/folders/9r/pt_53l_55hnfbqs4389kw3h40000gn/T/ipykernel_2708/2708157239.py", line 19, in <module>
    r = requests.get(WEATHER_URL, params = parameters)
  File "/Users/air/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/Users/air/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/Users/air/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/requests/sessions.py", line 529, in request
    resp = self.send(prep, **send_kwargs)
  File